## Настройка окружения

In [ ]:
%pip install -q transformers datasets

In [ ]:
%pip install -q pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 56.5 MB/s eta 0:00:00


In [ ]:
# Доля от общего размера датасета, которая будет поделена
# на тренировочную и тестовую части
DATASET_SIZE = 0.8

# Доля тестовой части от DATASET_SIZE
TEST_SIZE = 0.1

# Доля валидационной выборки от датасета, которая не была
# взята для тренировки и оценки, т.е. часть от доли (1 - DATASET_SIZE)
VAL_SIZE = 0.4

# Установка затравки для повторяемости результатов
SEED = 42

# Константа, которая определяет будет ли проводиться дообучение модели
ENABLE_FINE_TUNE = False

## Загрузка данных

In [ ]:
import pandas as pd
from datasets import Dataset
import ast

# Загружаем очищенный и подготовленный датасет
data = pd.read_csv(
    "./data/cleared/cleared_dataset.csv",
    sep=";",
    encoding='utf-8-sig',
    index_col=0
)
data["rubrics_list"] = data["rubrics_list"].apply(ast.literal_eval)
data.columns = ["rating", "rubrics_list", "text"]

# Формируем строку запроса для обучения модели
data['query'] = data.apply(
    lambda row: f"Рубрики: {', '.join(row['rubrics_list'])} | Рейтинг: {row['rating']} | Отзыв: ",
    axis=1
)

In [ ]:
data.dropna(inplace=True)

In [ ]:
# Вычислим длины запросов, чтобы определиться с параметрами токенизации
df_lens = data['query'].apply(lambda row: len(row))
# Выведем статистику
df_lens.describe()

,query
count,499800.000000
mean,61.422129
std,23.325686
min,35.000000
25%,42.000000
50%,54.000000
75%,74.000000
max,315.000000


In [ ]:
# Вычислим длины отзывов, чтобы определиться с параметрами токенизации
df_lens = data['text'].apply(lambda row: len(row))
# Выведем статистику
df_lens.describe()

,text
count,499800.000000
mean,307.633255
std,299.573765
min,2.000000
25%,146.000000
50%,218.000000
75%,369.000000
max,21213.000000


In [ ]:
# В данных есть очень длинные отзывы, но таких небольшое количество.
# На начальном этапе пока просто уберем их из обучения
data = data[df_lens <= 512]

In [ ]:
# Создание датасета Hugging Face
source_dataset = Dataset.from_pandas(data[['query', 'text']])

# Делим общий датасет на две части, одна будет использоваться для
# тернировки и тестирования, а другая для валидации
test_split_dataset = source_dataset.train_test_split(
    test_size=DATASET_SIZE,
    shuffle=True,
    seed=SEED,
)

# В данном случае "train" - это оставшаяся часть датасета (1 - DATASET_SIZE),
# из нее берем часть для валидационной выборки
val_dataset = test_split_dataset["train"].train_test_split(
    test_size=VAL_SIZE,
    shuffle=True,
    seed=SEED,
)

# Делим на тренировочную и тестовую части
dataset = test_split_dataset["test"].train_test_split(
    test_size=TEST_SIZE,
    shuffle=True,
    seed=SEED,
)

# Добавляем в dataset валидационный набор
dataset["validation"] = val_dataset["test"]

# Выводим информацию о полченном датасете
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'text', '__index_level_0__'],
        num_rows: 359856
    })
    test: Dataset({
        features: ['query', 'text', '__index_level_0__'],
        num_rows: 39984
    })
    validation: Dataset({
        features: ['query', 'text', '__index_level_0__'],
        num_rows: 39984
    })
})

In [ ]:
# Проверим запись из тренировочного набора
example = dataset['train'][11]

print("Запрос:", example["query"])
print("Отзыв:", example["text"])

Запрос: Рубрики: магазин канцтоваров, детские игрушки и игры, копировальный центр | Рейтинг: 5 | Отзыв: 
Отзыв: очень большой выбор. можно найти практически все. демократично цены. приветливый персонал. расположен в проходном месте. два входа. цокольный этаж. рекомендую для посещения.


In [ ]:
# Имя предобученного токенизатора
TOKENIZER_NAME = "cointegrated/rut5-small"

In [ ]:
from transformers import T5Tokenizer

# Загружаем с Hugging Face токенизатор
tokenizer = T5Tokenizer.from_pretrained(TOKENIZER_NAME)

# Максимальная длина входных данных
# Во входных данным мало символов, основная часть в рубриках, т.к.
# может быть перечислено несколько значений
max_input_length = 512

# Максимальная длина выходных данных (отзыва)
# Берем значение 512, т.к. более 75% отзывов имееют меньшую длину
max_target_length = 512

def preprocess_examples(examples):
  """Функция предобработки для токенизации
  """
  inputs = examples['query']
  texts = examples['text']

  model_inputs = tokenizer(
      inputs,
      max_length=max_input_length,
      padding="max_length",
      truncation=True
  )

  labels = tokenizer(
      texts,
      max_length=max_target_length,
      padding="max_length",
      truncation=True
  ).input_ids

  # Заменяем индекс токенов заполнения на -100,
  # чтобы они не учитывались в CrossEntropyLoss
  labels_with_ignore_index = []
  for labels_example in labels:
    labels_example = [label if label != 0 else -100 for label in labels_example]
    labels_with_ignore_index.append(labels_example)

  model_inputs["labels"] = labels_with_ignore_index

  return model_inputs

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

In [ ]:
# Проверим работу токенизатора на произвольном запросе
tokenizer.decode(
    tokenizer(
        data['query'].values[5000],
        max_length=max_input_length,
        padding="max_length",
        truncation=True
    ).input_ids
)

'<s>Рубрики: рынок, продуктовый рынок | Рейтинг: 5 | Отзыв: </s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [ ]:
# Теперь проверим работу токенизатора на произвольном отзыве
tokenizer.decode(
    tokenizer(
        data['text'].values[5000],
        max_length=max_target_length,
        padding="max_length",
        truncation=True
    ).input_ids
)

'<s>неплохой ассортимент, цены срелние, ближе к высоким, есть чистый и бесплатный туалет</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pa

In [ ]:
if ENABLE_FINE_TUNE:
    # Выполняем предобработку всех частей датасета
    dataset = dataset.map(preprocess_examples, batched=True)

Map:   0%|          | 0/359856 [00:00<?, ? examples/s]

Map:   0%|          | 0/39984 [00:00<?, ? examples/s]

Map:   0%|          | 0/39984 [00:00<?, ? examples/s]

In [ ]:
# Выведем информацию о полученном датасете
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 359856
    })
    test: Dataset({
        features: ['query', 'text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 39984
    })
    validation: Dataset({
        features: ['query', 'text', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 39984
    })
})

In [ ]:
from torch.utils.data import DataLoader

if ENABLE_FINE_TUNE:
    dataset.set_format(
        type="torch", columns=['input_ids', 'attention_mask', 'labels']
    )

    # Создаем объекты DataLoader с размерами батчей, чтобы они помещались в
    # видеопамяти. Значение batch_size=8 для тренировочной выборки было
    # подобрано для Nvidia RTX 4070Ti с 12 Гб видеопамяти. На этой карте
    # проводилось обучение модели.
    train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=8)
    valid_dataloader = DataLoader(dataset['validation'], batch_size=4)
    test_dataloader = DataLoader(dataset['test'], batch_size=4)

## Дообучение (fine-tune) модели с использованием PyTorch Lightning

In [ ]:
MODEL_NAME = "cointegrated/rut5-small"

In [ ]:
from transformers import (
    MT5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
)
import pytorch_lightning as pl

# Создаем класс модели с наследованием от pytorch_lightning.LightningModule
# PyTorch Lightning упрощает процесс обучения, в т.ч. следит за тем, чтобы
# все тензоры были на одном устройсте
class ReviewT5(pl.LightningModule):

    def __init__(self, lr=5e-5, num_train_epochs=15, warmup_steps=1000):
        super().__init__()
        self.model = MT5ForConditionalGeneration.from_pretrained(MODEL_NAME)
        self.save_hyperparameters()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        return outputs

    def common_step(self, batch, batch_idx):
        outputs = self(**batch)
        loss = outputs.loss

        return loss

    def training_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("training_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)
        self.log("validation_loss", loss, on_epoch=True)

        return loss

    def test_step(self, batch, batch_idx):
        loss = self.common_step(batch, batch_idx)

        return loss

    def configure_optimizers(self):

        optimizer = AdamW(self.parameters(), lr=self.hparams.lr)

        num_train_optimization_steps = self.hparams.num_train_epochs * len(train_dataloader)
        lr_scheduler = {
            'scheduler': get_linear_schedule_with_warmup(
                optimizer,
                num_warmup_steps=self.hparams.warmup_steps,
                num_training_steps=num_train_optimization_steps
                ),
            'name': 'learning_rate',
            'interval':'step',
            'frequency': 1
        }

        return {"optimizer": optimizer, "lr_scheduler": lr_scheduler}

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return valid_dataloader

    def test_dataloader(self):
        return test_dataloader

In [ ]:
if ENABLE_FINE_TUNE:
    # Создаем объект модели
    model = ReviewT5()

In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

# Каталог для сохранения результатов обучения
SAVE_DIRECTORY = "./t5_model"

# Обучение запускаем, если константа установлена в True
if ENABLE_FINE_TUNE:

    # Ранняя остановка обучения, если нет улучшений на валидации
    early_stop_callback = EarlyStopping(
        monitor='validation_loss',
        patience=3,
        strict=False,
        verbose=False,
        mode='min'
    )
    lr_monitor = LearningRateMonitor(logging_interval='step')

    trainer = Trainer(
        default_root_dir="./checkpoints",
        callbacks=[early_stop_callback, lr_monitor]
    )

    # Запускаем процесс обучения
    trainer.fit(model)

    # Сохраняем результаты в каталог
    model.model.save_pretrained(SAVE_DIRECTORY)

## Inference модели

In [ ]:
# Имя новой модели
REVIEW_MODEL_NAME = "kavlab/review-t5"

In [ ]:
if ENABLE_FINE_TUNE:
    # Загружаем веса модели из каталога
    model = MT5ForConditionalGeneration.from_pretrained(SAVE_DIRECTORY)
else:
    tokenizer = T5Tokenizer.from_pretrained(REVIEW_MODEL_NAME)
    model = MT5ForConditionalGeneration.from_pretrained(REVIEW_MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/639k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

In [ ]:
# Задаем произвольный индекс в исходном датасете
index = 2000

# Подготавливаем для подачи в модель
input_ids = tokenizer(
    data['query'].values[index], return_tensors='pt'
).input_ids

# Формируем отзыв
outputs = model.generate(
    input_ids,
    max_length=max_target_length,
    do_sample=True,
    temperature=0.7,
    repetition_penalty=1.2,
    num_beams=5,
    top_k=50,
    top_p=0.9,
)

print("Запрос:", data['query'].values[index])
print("Сгенирированый отзыв:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Исходный отзыв:", data['text'].values[index])

Запрос: Рубрики: солярий, салон красоты, парикмахерская | Рейтинг: 5 | Отзыв: 
Сгенирированый отзыв: Отличное место, приветливые администраторы, вежливые администраторы, всегда подскажут и помогут с выбором. Рекомендую!
Исходный отзыв: мой любимый салон, хожу в солярий только сюда уже много лет  приветливый и дружелюбный персонал  всем рекомендую 


In [ ]:
index = 2000

input_ids = tokenizer(
    data['query'].values[index], return_tensors='pt'
).input_ids

# Проверка с другими гиперпараметрами генерации
outputs = model.generate(
    input_ids,
    max_length=max_target_length,
    do_sample=True,
    temperature=0.9,
    repetition_penalty=1.5,
    num_beams=3,
    top_k=30,
    top_p=0.85,
)

print("Запрос:", data['query'].values[index])
print("Сгенирированый отзыв:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Исходный отзыв:", data['text'].values[index])

Запрос: Рубрики: солярий, салон красоты, парикмахерская | Рейтинг: 5 | Отзыв: 
Сгенирированый отзыв: Отличное место, всегда чисто и уютно, приветливый персонал. Приятная атмосфера, приветливые администраторы. Всегда предложат чай, кофе. Рекомендую!
Исходный отзыв: мой любимый салон, хожу в солярий только сюда уже много лет  приветливый и дружелюбный персонал  всем рекомендую 


In [ ]:
index = 2000

input_ids = tokenizer(
    data['query'].values[index], return_tensors='pt'
).input_ids

# Проверка с другими гиперпараметрами генерации
outputs = model.generate(
    input_ids,
    max_length=max_target_length,
    do_sample=True,
    temperature=0.85,
    repetition_penalty=1.5,
    num_beams=4,
    top_k=30,
    top_p=0.9,
    length_penalty=1.0,
)

print("Запрос:", data['query'].values[index])
print("Сгенирированый отзыв:", tokenizer.decode(outputs[0], skip_special_tokens=True))
print("Исходный отзыв:", data['text'].values[index])

Запрос: Рубрики: солярий, салон красоты, парикмахерская | Рейтинг: 5 | Отзыв: 
Сгенирированый отзыв: Прекрасное место! Очень приятный и вежливый персонал, всегда помогут и подскажут. Мастера знают свое дело и профессионалы своего дела. Рекомендую!
Исходный отзыв: мой любимый салон, хожу в солярий только сюда уже много лет  приветливый и дружелюбный персонал  всем рекомендую 


## Загрузка весов в hub Hugging Face

In [ ]:
# Если проводилось обучение, то загружаем результаты в hub
if ENABLE_FINE_TUNE:
    revision = "0.2.2"

    model.push_to_hub(
        REVIEW_MODEL_NAME,
        commit_message=f"based on {MODEL_NAME}",
        revision=revision,
    )

    tokenizer.push_to_hub(
        REVIEW_MODEL_NAME,
        commit_message=f"based on {TOKENIZER_NAME}",
        revision=revision,
    )